In [ ]:
# !pip install -r yolov7\requirements.txt

   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   -------------------------- ------------- 5.2/7.8 MB 26.5 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 22.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   -------------------- ------------------- 7.6/14.6 MB 36.2 MB/s eta 0:00:01
   ---------------------------------------- 14.6/14.6 MB 35.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   --- ------------------------------------ 3.7/38.8 MB 21.8 MB/s eta 0:00:02
   ----------- ---------------------------- 11.3/38.8 MB 30.7 MB/s eta 0:00:01
   ------------------- -------------------- 18.9/38.8 MB 32.2 MB/s eta 0:00:01
   -------------------------- ------------- 26.2/38.8 MB 33.2 MB/s eta 0:00:01
   ---------------------------------- ----- 33.8/38.8 MB 33.6 MB/s eta 0:00:01
   ---------------------------------------- 38.8/38.8 MB 32.5 MB/s eta 0:00:00
   ----

In [12]:
import os
import shutil

# 이미지가 저장된 폴더 경로
image_folder = 'E:/1.files/deeplearning/new_images/images'
# 이미지를 옮길 폴더 경로 (이미 만들어 놓은 폴더)
train_folder = 'E:/1.files/deeplearning/yolov7/data/train/images'
validation_folder = 'E:/1.files/deeplearning/yolov7/data/val/images'
test_folder = 'E:/1.files/deeplearning/yolov7/data/test/images'

# 이미지 파일 리스트 가져오기
images = os.listdir(image_folder)
images.sort()  # 정렬하여 일관성 있는 선택을 위해

# 각 데이터셋에 대한 리스트 초기화
train_images = []
validation_images = []
test_images = []

# 간격을 두고 선택
for i in range(len(images)):
    if i % 10 < 8:  # 0~7번째 인덱스는 train
        train_images.append(images[i])
    elif i % 10 == 8:  # 8번째 인덱스는 validation
        validation_images.append(images[i])
    else:  # 9번째 인덱스는 test
        test_images.append(images[i])

# 이미지 이동
for img in train_images:
    shutil.copy(os.path.join(image_folder, img), os.path.join(train_folder, img))
for img in validation_images:
    shutil.copy(os.path.join(image_folder, img), os.path.join(validation_folder, img))
for img in test_images:
    shutil.copy(os.path.join(image_folder, img), os.path.join(test_folder, img))

print("이미지가 성공적으로 분리되었습니다.")


이미지가 성공적으로 분리되었습니다.


In [13]:
#task 정의에 따라 label_set들을 class를 0부터 시작하는 정수 번호로 지정해주는 함수를 정의한다.
def class_choose(cls):
    paper = ['c_1','c_2_01','c_2_02', 'c_1_01', 'c_2_02_01']
    glass = ['c_4_01_02','c_4_02_01_02','c_4_02_02_02','c_4_02_03_02','c_4_03', 'c_4_03_01', 'c_4_01_01', 'c_4_02_01_01', 'c_4_02_02_01', 'c_4_02_03_01']
    pet = ['c_5_02', 'c_5_01_01', 'c_5_02_01', 'c_5_01']
    plastic =['c_6', 'c_6_01']
    can = ['c_3', 'c_3_01']
    vinyl =['c_7', 'c_7_01']
    battery =['c_9']
    styro=['c_8_01','c_8_02', 'c_8_01_01']

    if cls in paper:
        return 0
    elif cls in glass:
        return 1
    elif cls in pet:
        return 2
    elif cls in plastic:
        return 3
    elif cls in can:
        return 4
    elif cls in vinyl:
        return 5
    elif cls in battery:
        return 6
    elif cls in styro:
        return 7

In [14]:
import json
import cv2
import os
from tqdm import tqdm

def resize_and_label(image_dir, output_label_dir):
    # 통일할 이미지 크기 설정 (예: 1280x1280)
    target_size = (640, 640)

    # 이미지 및 레이블 디렉토리 경로 설정
    label_dir = 'E:/1.files/deeplearning/new_images/labels'  # JSON 레이블 디렉토리

    # 이미지 파일 이름 목록 생성
    names = [os.path.splitext(file)[0] for file in os.listdir(image_dir) if file.lower().endswith('.jpg')]
    print(len(names))

    for file in tqdm(names, desc="처리 진행 상황"):
        json_path = os.path.join(label_dir, file + '.json')
        img_path = os.path.join(image_dir, file + '.jpg')
        image = cv2.imread(img_path)
        if image is None:  # 이미지가 제대로 읽히지 않은 경우
            print(f"이미지를 읽을 수 없습니다: {img_path}")
            continue
        
        # 원본 이미지 크기 가져오기
        h, w, _ = image.shape

        # 이미지 크기 조정
        resized_image = cv2.resize(image, target_size)
        resized_h, resized_w = target_size  # 조정된 이미지 크기

        # JSON 파일 열기
        with open(json_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)

        objs = []
        for obj in json_data['objects']:
            if obj['annotation_type'] != 'box':  # bbox가 아닌 polygon 등 제외
                continue
            cls = obj['class_name']
            cls_num = class_choose(cls)  # 클래스 숫자로 변환하는 함수 (직접 구현 필요)

            # JSON 파일에서 x, y, width, height 값 가져오기
            x = obj['annotation']['coord']['x']
            y = obj['annotation']['coord']['y']
            width = obj['annotation']['coord']['width']
            height = obj['annotation']['coord']['height']

            # 좌표 변환 비율 계산
            width_ratio = target_size[0] / w
            height_ratio = target_size[1] / h

            # 좌표를 변환된 이미지 크기에 맞게 조정
            x1 = x * width_ratio
            y1 = y * height_ratio
            x2 = (x + width) * width_ratio
            y2 = (y + height) * height_ratio

            # YOLO 포맷으로 변환
            center_x = (x1 + x2) / (2 * resized_w)
            center_y = (y1 + y2) / (2 * resized_h)
            width_norm = (x2 - x1) / resized_w
            height_norm = (y2 - y1) / resized_h

            objs.append(f'{cls_num} {center_x} {center_y} {width_norm} {height_norm}\n')

        # YOLO 형식의 라벨 파일 저장
        output_path = os.path.join(output_label_dir, f'{file}.txt')
        with open(output_path, 'w') as f:
            f.writelines(objs)

        # 조정된 이미지 저장 (원하는 경우)
        output_image_path = os.path.join(image_dir, f'{file}.jpg')  # 원래 경로에 저장하려면 주석 해제
        cv2.imwrite(output_image_path, resized_image)

    print("변환이 완료되었습니다.")


In [15]:
import os

def compare_folders(a_folder, b_folder):
    """
    두 폴더(a_folder, b_folder) 내의 파일 이름(확장자 제외)이 동일한지 확인하는 함수.
    
    Parameters:
    - a_folder: 첫 번째 폴더 경로.
    - b_folder: 두 번째 폴더 경로.
    
    Returns:
    - 동일한 파일 이름이 있는지 여부를 반환.
    """
    
    # 각 폴더의 파일 목록 가져오기 (확장자 제외한 파일 이름만)
    a_files = {os.path.splitext(f)[0] for f in os.listdir(a_folder)}
    b_files = {os.path.splitext(f)[0] for f in os.listdir(b_folder)}
    
    # 두 폴더의 파일 이름 비교
    if a_files == b_files:
        print("두 폴더의 파일 이름이 동일합니다.")
        return True
    else:
        # 파일 이름 차이점 출력
        only_in_a = a_files - b_files
        only_in_b = b_files - a_files
        
        if only_in_a:
            print(f"a 폴더에만 있는 파일들: {only_in_a}")
        if only_in_b:
            print(f"b 폴더에만 있는 파일들: {only_in_b}")
        
        return False

# 사용 예시
a_folder = "E:/1.files/deeplearning/new_images/images"
b_folder = "E:/1.files/deeplearning/new_images/labels"

compare_folders(a_folder, b_folder)


두 폴더의 파일 이름이 동일합니다.


True

In [16]:
image_dir = 'E:/1.files/deeplearning/yolov7/data/test/images'  # 이미지 디렉토리
output_label_dir = 'E:/1.files/deeplearning/yolov7/data/test/labels' # YOLO 형식으로 변환한 레이블 저장 경로
resize_and_label(image_dir, output_label_dir)

image_dir = 'E:/1.files/deeplearning/yolov7/data/train/images'  # 이미지 디렉토리
output_label_dir = 'E:/1.files/deeplearning/yolov7/data/train/labels' # YOLO 형식으로 변환한 레이블 저장 경로
resize_and_label(image_dir, output_label_dir)

image_dir = 'E:/1.files/deeplearning/yolov7/data/val/images'  # 이미지 디렉토리
output_label_dir = 'E:/1.files/deeplearning/yolov7/data/val/labels' # YOLO 형식으로 변환한 레이블 저장 경로
resize_and_label(image_dir, output_label_dir)


363


처리 진행 상황: 100%|██████████| 363/363 [00:31<00:00, 11.57it/s]


변환이 완료되었습니다.
2908


처리 진행 상황: 100%|██████████| 2908/2908 [05:42<00:00,  8.49it/s]


변환이 완료되었습니다.
363


처리 진행 상황: 100%|██████████| 363/363 [00:43<00:00,  8.27it/s]

변환이 완료되었습니다.


In [17]:
import os
from collections import defaultdict

def count_objects_by_class(label_folder):
    """
    모든 라벨 파일을 순회하며 클래스별 객체 개수를 세는 함수.
    
    Parameters:
    - label_folder: 라벨 파일이 저장된 경로.
    
    Returns:
    - class_counts: 클래스 ID를 키로 하고, 각 클래스별 객체 수를 값으로 가지는 딕셔너리.
    """
    
    # 클래스별 객체 수를 저장할 딕셔너리 초기화
    class_counts = defaultdict(int)
    
    # 라벨 폴더 내 모든 파일을 순회
    for label_file in os.listdir(label_folder):
        label_path = os.path.join(label_folder, label_file)
        
        # 라벨 파일 읽기
        with open(label_path, 'r') as file:
            lines = file.readlines()
            
            for line in lines:
                # 객체 정보를 YOLO 형식으로 파싱
                parts = line.strip().split()
                class_id = int(parts[0])
                
                # 해당 클래스 ID의 객체 수 증가
                class_counts[class_id] += 1
    
    return class_counts

# 사용 예시
label_folder = "path/to/labels"
class_counts = count_objects_by_class('E:/1.files/deeplearning/yolov7/data/train/labels')

# 클래스별 객체 개수 출력
for class_id, count in class_counts.items():
    print(f"Class {class_id}: {count} objects")

Class 1: 896 objects
Class 4: 983 objects
Class 2: 817 objects
Class 3: 1122 objects
Class 5: 938 objects
Class 0: 1336 objects
Class 6: 782 objects
Class 7: 813 objects


In [20]:
import os

def check_empty_labels(label_folder):
    """
    주어진 폴더 내의 YOLO 라벨 텍스트 파일들이 비어있는지 확인하고 출력합니다.

    Parameters:
    - label_folder: 라벨 파일들이 들어 있는 폴더 경로.
    """
    
    # 라벨 파일들이 들어 있는 폴더 내의 모든 파일을 확인
    empty_files = []  # 비어있는 파일 목록
    
    for filename in os.listdir(label_folder):
        # 텍스트 파일만 필터링
        if filename.endswith(".txt"):
            file_path = os.path.join(label_folder, filename)
            
            # 파일이 비어있는지 확인
            if os.path.getsize(file_path) == 0:
                empty_files.append(filename)
    
    # 결과 출력
    if empty_files:
        print(f"비어있는 라벨 파일들: {empty_files}")
    else:
        print("비어있는 라벨 파일이 없습니다.")

# 사용 예시
label_folder = "yolov7/data/val/labels"
check_empty_labels(label_folder)


비어있는 라벨 파일이 없습니다.


In [23]:
import os

def check_invalid_coordinates(label_folder):
    """
    주어진 폴더 내의 YOLO 라벨 파일들이 유효한지 확인하고 잘못된 좌표값을 찾습니다.

    Parameters:
    - label_folder: 라벨 파일들이 들어 있는 폴더 경로.
    """
    
    # 라벨 파일들이 들어 있는 폴더 내의 모든 파일을 확인
    invalid_files = []  # 잘못된 좌표값을 가진 파일 목록
    
    for filename in os.listdir(label_folder):
        # 텍스트 파일만 필터링
        if filename.endswith(".txt"):
            file_path = os.path.join(label_folder, filename)
            
            with open(file_path, "r") as f:
                lines = f.readlines()
                
                for line in lines:
                    # 각 라벨 줄을 공백 기준으로 나눔
                    values = line.strip().split()
                    
                    # 값이 5개인지 확인 (class x_center y_center width height)
                    if len(values) == 5:
                        try:
                            # 각 값 파싱
                            cls, x_center, y_center, width, height = map(float, values)
                            
                            # 클래스 값 확인 (양의 정수)
                            if cls < 0 or not cls.is_integer():
                                invalid_files.append(f"{filename} (Invalid class: {cls})")
                            
                            # 좌표 값 확인 (0 ~ 1 사이의 값이어야 함)
                            if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= width <= 1 and 0 <= height <= 1):
                                invalid_files.append(f"{filename} (Invalid coordinates: {values})")
                            
                        except ValueError:
                            # 값이 실수로 변환되지 않으면 잘못된 라벨로 처리
                            invalid_files.append(f"{filename} (Invalid value format)")
    
    # 결과 출력
    if invalid_files:
        print("잘못된 좌표값을 가진 파일들:")
        for file in invalid_files:
            print(file)
    else:
        print("모든 라벨 파일이 유효합니다.")

# 사용 예시
label_folder = "yolov7/data/test/labels"
check_invalid_coordinates(label_folder)


모든 라벨 파일이 유효합니다.
